# GQA 分组查询注意力

**GQA（Grouped-Query Attention）** 是Transformer中多头注意力机制的优化变体，它通过将多个查询头进行分组，并在每组内共享同一组键和值，从而在降低计算成本和内存占用的同时，较好地平衡了模型的性能和效率，尤其适用于长文本处理和大规模推理任务。

## 详细原理图

![GQA Struct](img/GQA-Struct.svg)

---

## 1. 代码实现

---

#### 引入必要的库

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import math

#### KV 复制函数

由于我们对注意力计算中的 Q、K、V 进行了分组，而每个分组中是由多个 Q 对应一个 KV，那么在做矩阵运算时，就会出现维度不一致的问题，这时我们就需要对 KV 进行复制，使其在维度上保持一致。如下图所示：

![KV Repeat](img/KV-Repeat.svg)

下面是 KV 复制函数的实现代码，其中 pytorch 也提供了 `torch.repeat_interleave(input,repeats,dim,output_size)` 函数用于张量的复制和扩充。

In [ ]:
def repeat_kv(x, n_rep):
    """
    复制并扩充 KV 矩阵，等价于 `torch.repeat_interleave(x, dim=2, repeats=n_rep)`

    :param x: 传入的数据
    :param n_rep:  重复的数量
    :return: 复制完成的矩阵
    """
    batch, seq_len, kv_head_num, dim_head = x.size()
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(batch, seq_len, kv_head_num, n_rep, dim_head)
        .reshape(batch, seq_len, kv_head_num * n_rep, dim_head)
    )

#### 分组注意力代码实现

这里的实现将不再关注 Decoder 的传入数据，即实现的是自注意力机制。

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, dim_emb:int, dim_head:int, dim_out:int, q_head_num:int, kv_head_num:int):
        """
        Grouped Query Attention

        :param dim_emb: 嵌入维度
        :param dim_head: 头维度
        :param dim_out: 输出维度
        :param q_head_num: Q 头数
        :param kv_head_num: KV 头数
        """

        super().__init__()
        self.dim_emb = dim_emb
        self.dim_head = dim_head
        self.dim_out = dim_out
        self.q_head_num = q_head_num
        self.kv_head_num = kv_head_num

        # 分组的数量
        self.group_num = q_head_num // kv_head_num

        # 创建 Attention 权重
        self.Q_net = nn.Linear(dim_emb, q_head_num * dim_head)
        self.K_net = nn.Linear(dim_emb, kv_head_num * dim_head)
        self.V_net = nn.Linear(dim_emb, kv_head_num * dim_head)

        # 创建权重
        self.W_net = nn.Linear(q_head_num * dim_head, dim_out)

    def forward(self, x, mask_mat = None, use_cache = False, kv_cache = None):

        batch,seq_len,_ = x.size()

        # 计算 qkv 的值
        # q   的维度为 (batch, seq_len, q_head_num  , dim_head)
        # k/v 的维度为 (batch, seq_len, kv_head_num , dim_head)
        q = self.Q_net(x).view(batch, seq_len, self.q_head_num , self.dim_head)
        k = self.K_net(x).view(batch, seq_len, self.kv_head_num, self.dim_head)
        v = self.V_net(x).view(batch, seq_len, self.kv_head_num, self.dim_head)

        # q_head_num 与 kv_head_num 是不一致的，将其复制后使其维度保持一致
        # qkv 的维度为 (batch, q_head_num, seq_len, dim_head)
        q = q.transpose(1, 2)
        k = repeat_kv(k, self.group_num).transpose(1, 2)
        v = repeat_kv(v, self.group_num).transpose(1, 2)

        # 如果使用 KV Cache，拼接历史 K 和 V
        if use_cache and kv_cache is not None:
            k_prev, v_prev = kv_cache
            # 在序列长度维度拼接
            k = torch.cat([k_prev, k], dim=2)
            v = torch.cat([v_prev, v], dim=2)

        # 记录新的 KV Cache（供下一步使用）
        new_kv_cache = (k, v) if use_cache else None

        # 将 K 的最后两个维度进行转置，转置后的维度为 (batch, q_head_num, dim_head, seq_len)
        k_t = k.transpose(-1, -2)

        # 计算 qk^T / sqrt(d_k)，此时 s0 的维度为 (batch, q_head_num, seq_len, seq_len)
        s0 = torch.matmul(q, k_t) / math.sqrt(self.q_head_num)

        # 进行掩码遮掩操作
        if mask_mat is not None:
            # 进行遮掩
            s0 = torch.masked_fill(
                s0,
                mask_mat,
                float('-inf')
            )

        # 计算 softmax(s)*v ，此时 s1 的维度为 (batch, q_head_num, seq_len, dim_head)
        s1 = torch.matmul(F.softmax(s0, dim=-1), v)

        # 我们需要使用 s1*W ，这里就要将矩阵变换为可以跟 W 矩阵进行矩阵乘法的维度，即：
        # s1 变换维度为：(batch, seq_len, dim_head * q_head_num)
        s1 = s1.transpose(1, 2).contiguous() # 这里需要让内存连续（ 使用 reshape 则不用）
        s1 = s1.view(batch, seq_len, self.q_head_num * self.dim_head)

        # 输出的最终维度为：(batch, seq_len, dim_out)
        output = self.W_net(s1)

        return output, new_kv_cache

## 模型测试

---

In [ ]:
# 配置
# =====================================
batch = 3
seq_len = 8
dim_emb = 72
q_head_num = 24
kv_head_num = 6
dim_head = dim_emb // q_head_num
dim_out = dim_emb
# =====================================

attention = GroupedQueryAttention(dim_emb, dim_head, dim_emb, q_head_num, kv_head_num)

# ===============
#  模拟第一次调用
# ===============

# 模拟第一次传入的 X，这个 seq_len 可以不用限制
X = torch.randn((batch, seq_len, dim_emb))

# 生成不包含对角线的上三角矩阵（设置 diagonal=1）
mask_mat = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)

# 第一次输出
output, kv_cache = attention(X, mask_mat, use_cache = True)
k_cache, v_cache = kv_cache

# 打印输出及 KV Cache 维度信息
print(f'output size : {output.size()}')
print(f'k cache size : {k_cache.size()}')
print(f'v cache size : {v_cache.size()}')

# ===============
#  模拟第二次调用
# ===============

# 模拟生成下一个 token
# 注意此时需要 seq = 1，因为使用 KV Cache 只需传入下一个生成的 token 即可
X_next = torch.randn(batch, 1, dim_emb)

# 第二次调用（携带缓存），此时不用传入掩码矩阵
output_next, kv_cache = attention(X_next, use_cache=True, kv_cache=kv_cache)
k_cache_next, v_cache_next = kv_cache

# 打印第二次调用的输出及 KV Cache 维度信息
print(f'output size : {output_next.size()}')
print(f'next k cache size : {k_cache_next.size()}')
print(f'next v cache size : {v_cache_next.size()}')